# Invisible Light - plotting a galaxy's colours

In this notebook, you will be able to use data from the LOFAR deep fields release to choose a galaxy and plot its _spectral energy distribution_ (SED)- that is, how bright the galaxy is across the electromagnetic spectrum. This data is from a well-studied field called the Lockman Hole, which is a patch of sky located in the constellation of Ursa Major. 

The following exercises assume some experience programming in Python - if you have never used Python before, don't worry! Have a look at the solutions and see if you can understand what they do, and try changing some variables to see how it works.

If you would like a quick introduction to using Jupyter, you can click "Help" -> "User interface tour" which will show you how this works, and you can read about running code in the notebook [here](https://nbviewer.jupyter.org/github/ipython/ipython/blob/3.x/examples/Notebook/Running%20Code.ipynb).

First we will import the Python libraries that we will use in this exercise:


In [ ]:
from astropy.io import fits              # astropy is a library of helpful tools for astronomy
from astropy import units as u           # for example, units helps us keep track of astronomical units in calculations
from astropy.coordinates import SkyCoord # ...and SkyCoord can do transformations with astronomical coordinates
import matplotlib.pyplot as plt          # matplotlib is library for plotting 
from scipy.interpolate import interp1d   # scipy has useful maths tools, such as this curve fitting tool
import numpy as np                       # numpy is for storing & manipulating data in arrays

Then we will import the data files. All of the data is publicly available on the LOFAR surveys website, but the data files are very large (> 1 GB!) and take a long time to load, so we're using a short version of the table that I put together for my research. 

In astronomy, data is often stored in a file format called "fits" and one way to access it is to use the Astropy libary we imported above, which has a "fits" function to open such files.


In [ ]:
data_file = fits.open("https://star.herts.ac.uk/~joanna/LOFAR_galaxies.fits")

table = data_file[1].data
headers = data_file[1].header.keys

# Now we have two variables:
# - table, which contains all of the table data
# - header, which just contains the information in the table header i.e. the names of the columns

This data file is a table of data, with fluxes at several different wavelengths across the electromagnetic spectrum, from low frequency radio in LOFAR all the way through to optical and UV. If you print the variable "headers", you can see the names of all of the columns in the table. Some of these are a bit cryptic! They will be explained later.


In [ ]:
# Try printing "headers"


Each row of the table is a different galaxy, and you can find its position on the sky by looking at the right ascension (RA) and declination (Dec) in the table, which are given with the keys 'RA' and 'DEC'.

Look up the coordinates of some of the galaxies in the table in [Aladin](https://aladin.u-strasbg.fr/AladinLite/)  - this tool shows sky surveys at multiple wavelengths, so you can see what your galaxy looks like across the spectrum.




In [ ]:
# Each row of the table holds the data for a single galaxy, so you should start by selecting a single row.
# Inside each row, you can access the columns as follows: column1 = table[row number]["name of column 1"]
# but replacing the row number and column names with the correct values. You can find some other column names
# in "header" in the previous cell.
#
# Complete the code below to find the RA and Dec of a galaxy in the table:

galaxy = 


RA = 
Dec = 

# SkyCoord can translate these coordinates from degrees (their units in the table) to hours minutes and seconds,
# if you want to view them like that. Aladin can read several different formats - the one below included:

coords = SkyCoord(RA*u.degree, Dec*u.degree)

print(f"Coordinates of galaxy {n}: {coords.ra.hms[0]:0.0f} {coords.ra.hms[1]:0.0f} {coords.ra.hms[2]:0.2f}, \
 +{coords.dec.dms[0]:0.0f} {coords.dec.dms[1]:0.0f} {coords.dec.dms[2]:0.2f}")

# The long line of code above is just to nicely format our coordinates


Here is a table that shows the different "bands" of the observations, the name of the column with the data for that band in the table, the units for the data in that column, and the wavelength that the band corresponds to.


<!-- "ugriz" is short for U-band, G-band, R-band, I-band, and Z-band, detailed on [the Wikipedia article on photometric systems](https://en.wikipedia.org/wiki/Photometric_system): -->
$$\begin{array}{|c|c|c|}
\hline \text{Band}                  & \text{Table key}        & \text{Units}       & \text{Wavelength}(\lambda)\\
\hline \text{u}                     & \text{'u_flux'}         & \text{microJy}     & \text{365 nm}\\
\hline \text{g}                     & \text{'g_flux'}         & \text{microJy}     & \text{475 nm}\\
\hline \text{r}                     & \text{'r_flux'}         & \text{microJy}     & \text{658 nm}\\
\hline \text{i}                     & \text{'i_rcs_flux'}     & \text{microJy}     & \text{806 nm}\\
\hline \text{z}                     & \text{'z_flux'}         & \text{microJy}     & \text{900 nm}\\
\hline \text{J}                     & \text{'J_flux'}         & \text{microJy}     & \text{1.24}\mu\text{m} \\
\hline \text{K}                     & \text{'K_flux'}         & \text{microJy}     & \text{2.19}\mu\text{m} \\
\hline \text{SWIRE channel 1}       & \text{'ch1_swire_flux'} & \text{microJy}     & \text{3.6}\mu\text{m} \\
\hline \text{SWIRE channel 2}       & \text{'ch2_swire_flux'} & \text{microJy}     & \text{4.5}\mu\text{m} \\
\hline \text{SWIRE channel 3}       & \text{'ch3_swire_flux'} & \text{microJy}     & \text{5.8}\mu\text{m} \\
\hline \text{SWIRE channel 4}       & \text{'ch4_swire_flux'} & \text{microJy}     & \text{8}\mu\text{m} \\
\hline \text{Spitzer MIPS}          & \text{'F_MIPS_24'}      & \text{microJy}     & \text{24}\mu\text{m} \\
\hline \text{Herschel PACS band 1}  & \text{'F_PACS_100'}     & \text{mJy}         & \text{100}\mu\text{m} \\
\hline \text{Herschel PACS band 2}  & \text{'F_PACS_160'}     & \text{mJy}         & \text{160}\mu\text{m} \\
\hline \text{Herschel Spire band 1} & \text{'F_SPIRE_250'}    & \text{mJy}         & \text{250}\mu\text{m} \\
\hline \text{Herschel Spire band 2} & \text{'F_SPIRE_350'}    & \text{mJy}         & \text{350}\mu\text{m} \\
\hline \text{Herschel Spire band 3} & \text{'F_SPIRE_500'}    & \text{mJy}         & \text{500}\mu\text{m} \\
\hline
\end{array}$$


In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~
# From the information in the table above, make a list of fluxes from the table:
#~~~~~~~~~~~~~~~~~~~~~~~~
fluxes = []

#~~~~~~~~~~~~~~~~~~~~~~~~
# Remember the units! Not all of the fluxes are in the same units. Can you correct for that with some code?
#~~~~~~~~~~~~~~~~~~~~~~~~


#~~~~~~~~~~~~~~~~~~~~~~~~
# You will also need a list of wavelengths at which these fluxes are measured:
#~~~~~~~~~~~~~~~~~~~~~~~~
wavelengths = []





Now plot the SED: wavelength on the x-axis, and flux on the y-axis. You might find it helpful to plot it on a log scale. Can you plot or fit a line between the points to show the shape of the SED more clearly?

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~
# Write the code to plot the SED in this box. You can use matplotlib.
#~~~~~~~~~~~~~~~~~~~~~~~~



#~~~~~~~~~~~~~~~~~~~~~~~~

plt.show()


Compare the shape of the SED of this galaxy to some typical SEDs of different types of galaxies below. What type of galaxy do you think this is? Are there any other ways we could confirm that?

Note: the x-axis here shows _spectral flux density_, not just flux. That means flux per unit wavelength, so to make our plot look the same, we need to divide our fluxes by wavelength. Modify your code above to do this.

![text](typical_SEDs.jpg)